In [950]:
from qoop.core import state, ansatz, metric
import qiskit
from qiskit import transpile
from qiskit.quantum_info import Operator, DensityMatrix, Kraus
from scipy.linalg import qr
import numpy as np
import tensorflow as tf

<img src = './docs/1.png' height ='800px'>

In [981]:
# Step 1: Read quantum compilation and learn how to use qoop
# https://github.com/vutuanhai237/qoop/wiki/Advance:-Custom-state-preparation

# Step 2: Implement the following function

#Create a ansatz V with n qubits
def V(num_qubits: int):
    return ansatz.stargraph (num_qubits=num_qubits)

# def Epsilon(rho, kraus_operators):
#     # K = K_noise = [\sqrt(p) I @ I, \sqrt(1-p) Z @ Z]
#     # see Eq. 1 Ref. [1]
#     return sum(K @ rho.data @ np.transpose(np.conjugate(K)) for K in kraus_operators)

# def Epsilon2(rho, unitary_matrix):
#     # K = K_noise = [\sqrt(p) I @ I, \sqrt(1-p) Z @ Z]
#     # see Eq. 1 Ref. [1]
#     return (np.transpose(np.conjugate(unitary_matrix)) @ rho.data @ unitary_matrix)

def calRho3 (rho, unitary_matrix, kraus_operators):
    rho2 = sum(K @ rho @ np.transpose(np.conjugate(K)) for K in kraus_operators)
    rho3 = (np.transpose(np.conjugate(unitary_matrix)) @ rho2 @ unitary_matrix)
    return rho3

def createKraus(unitary_matrix):
    kraus_ops = []
    Q, R = qr(unitary_matrix)

    #print(Q.shape)
    for q in Q:
        
        q = np.expand_dims(q, 1)
        #print(q)
        #print(np.transpose(np.conjugate(q)))
        #print(q @ np.transpose(np.conjugate(q)))
        kraus_ops.append(q @ np.transpose(np.conjugate(q)))

    
    return tf.convert_to_tensor(kraus_ops)

def tf_sqrtm(matrix):
    # Eigenvalue Decomposition to compute the matrix square root
    eigenvalues, eigenvectors = tf.linalg.eigh(matrix)
    sqrt_eigenvalues = tf.sqrt(tf.maximum(eigenvalues, 0))  # Ensure non-negative eigenvalues
    sqrt_matrix = tf.matmul(eigenvectors, tf.linalg.diag(sqrt_eigenvalues))
    sqrt_matrix = tf.matmul(sqrt_matrix, tf.linalg.adjoint(eigenvectors))
    return sqrt_matrix

def compilation_trace_fidelity(rho, sigma):
    """Calculating the fidelity metric

    Args:
        - rho (DensityMatrix): first density matrix
        - sigma (DensityMatrix): second density matrix

    Returns:
        - float: trace metric has value from 0 to 1
    """
    rho2 = tf.linalg.sqrtm((tf.linalg.sqrtm(rho)) @ (rho))

    # Cast to a supported type
    real_part = tf.math.real(rho2)
    imaginary_part = tf.math.imag(rho2)

    # Check for NaNs in both real and imaginary parts
    contains_nan_real = tf.reduce_any(tf.math.is_nan(real_part))
    contains_nan_imag = tf.reduce_any(tf.math.is_nan(imaginary_part))

    contains_nan = contains_nan_real or contains_nan_imag

    if contains_nan == True:
        rho2 = rho

    return tf.linalg.trace(
            rho2
            @ (tf.linalg.sqrtm(sigma))
        )
    


In [952]:
n = 2

#Create a ansatz V with n qubits
circuit = V(n)

#Assign random parameter
num_params = circuit.num_parameters
x0 = 2 * np.pi * np.random.random(num_params)
circuit = circuit.assign_parameters(dict(zip(circuit.parameters, x0)))

print(circuit)

     ┌────────────┐   
q_0: ┤ Ry(1.6548) ├─■─
     ├────────────┤ │ 
q_1: ┤ Ry(5.2894) ├─■─
     └────────────┘   


In [953]:
# Get the unitary operator corresponding to the circuit
unitary_op = Operator(circuit)

# Get the unitary matrix
unitary_matrix = unitary_op.data
print (unitary_matrix.shape) #(16, 16 as num_qubits is 4)

(4, 4)


In [954]:
KrausOperators = createKraus(unitary_matrix=unitary_matrix)
print(sum(K @ np.transpose(np.conjugate(K)) for K in KrausOperators))


tf.Tensor(
[[ 1.00000000e+00+0.j  1.11022302e-16+0.j  2.77555756e-17+0.j
   1.11022302e-16+0.j]
 [ 1.11022302e-16+0.j  1.00000000e+00+0.j  0.00000000e+00+0.j
   0.00000000e+00+0.j]
 [ 2.77555756e-17+0.j  0.00000000e+00+0.j  1.00000000e+00+0.j
  -2.22044605e-16+0.j]
 [ 1.11022302e-16+0.j  0.00000000e+00+0.j -2.22044605e-16+0.j
   1.00000000e+00+0.j]], shape=(4, 4), dtype=complex128)


In [955]:
print(KrausOperators)

tf.Tensor(
[[[ 0.35396813+0.j -0.38501652+0.j  0.19195022+0.j -0.20878718+0.j]
  [-0.38501652+0.j  0.41878832+0.j -0.20878718+0.j  0.22710099+0.j]
  [ 0.19195022+0.j -0.20878718+0.j  0.10409098+0.j -0.11322134+0.j]
  [-0.20878718+0.j  0.22710099+0.j -0.11322134+0.j  0.12315257+0.j]]

 [[ 0.41878832+0.j  0.38501652+0.j  0.22710099+0.j  0.20878718+0.j]
  [ 0.38501652+0.j  0.35396813+0.j  0.20878718+0.j  0.19195022+0.j]
  [ 0.22710099+0.j  0.20878718+0.j  0.12315257+0.j  0.11322134+0.j]
  [ 0.20878718+0.j  0.19195022+0.j  0.11322134+0.j  0.10409098+0.j]]

 [[ 0.10409098+0.j -0.11322134+0.j -0.19195022+0.j  0.20878718+0.j]
  [-0.11322134+0.j  0.12315257+0.j  0.20878718+0.j -0.22710099+0.j]
  [-0.19195022+0.j  0.20878718+0.j  0.35396813+0.j -0.38501652+0.j]
  [ 0.20878718+0.j -0.22710099+0.j -0.38501652+0.j  0.41878832+0.j]]

 [[ 0.12315257+0.j  0.11322134+0.j -0.22710099+0.j -0.20878718+0.j]
  [ 0.11322134+0.j  0.10409098+0.j -0.20878718+0.j -0.19195022+0.j]
  [-0.22710099+0.j -0.20878718+

In [956]:
#rho
rho = DensityMatrix.from_label('0' * n)
rho = tf.convert_to_tensor(rho)
rho3 = calRho3(rho=rho, kraus_operators=KrausOperators, unitary_matrix=unitary_matrix) 
print(rho3)
print(rho)


tf.Tensor(
[[ 0.31532961+0.j  0.0919414 +0.j  0.07497501+0.j -0.07359183+0.j]
 [ 0.0919414 +0.j  0.26449039+0.j -0.05448421+0.j  0.05060549+0.j]
 [ 0.07497501+0.j -0.05448421+0.j  0.19676842+0.j -0.09649031+0.j]
 [-0.07359183+0.j  0.05060549+0.j -0.09649031+0.j  0.22341158+0.j]], shape=(4, 4), dtype=complex128)
tf.Tensor(
[[1.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]], shape=(4, 4), dtype=complex128)


In [968]:
#cost func compare rho, rho3
def cost(rho, rho3):
    return 1-compilation_trace_fidelity(rho, rho3)

print(cost(rho=rho, rho3=rho3))

#Nhận U, K trả về rho3

tf.Tensor((0.45678794755065755+0j), shape=(), dtype=complex128)


In [958]:
# Step 3: Implement the following function
# state_need_tomogaphy = ...
# rho = np.conjugate(np.transpose(state_need_tomogaphy)) @ state_need_tomogaphy   # density matrix
# rho' = Delta(rho)
# compiler = qoop.qcompilation.QuantumCompilation(U = rho', V = V())
# compiler.fit()
# compiler.plot()
# see fidelities versus iteration

<img src = './docs/2.png' height ='800px'>

<img src = './docs/3.jpg' height ='800px'>

<img src = './docs/4.png' height ='200px'>

In [959]:
"""def Derivative(rho, unitary_matrix, kraus_operators, epsilon=0.01, alpha=0.1):
    # Deep copy of the Kraus operators to avoid unintended modifications
    kraus_operators_plus = kraus_operators.copy()
    kraus_operators_minus = kraus_operators.copy()
    c = []
    
    # Loop through Kraus operators
    for n, k in enumerate(kraus_operators):
        k_plus = k + epsilon # Kj(+epsilon)
        k_minus = k - epsilon # Kj(-epsilon)
      
        kraus_operators_plus[n] = k_plus # Replace Kj by new Kj_plus
        kraus_operators_minus[n] = k_minus # Replace Kj by new Kj_minus

        # Compute fidelity for K+ and K-
        fil_plus = metric.compilation_trace_fidelity(rho, calRho3(rho=rho, unitary_matrix=unitary_matrix, kraus_operators=kraus_operators_plus))
        fil_minus = metric.compilation_trace_fidelity(rho, calRho3(rho=rho, unitary_matrix=unitary_matrix, kraus_operators=kraus_operators_minus))
        
        # Compute the derivative
        derivative = -(fil_plus - fil_minus) / epsilon

        # Accumulate
        c.append(derivative)
        
        # Reset Kraus operators for next iteration
        kraus_operators_plus[n] = k
        kraus_operators_minus[n] = k
    
    # Convert c to numpy array for further calculations
    c = np.array(c)
    
    # Compute the projection term
    kraus_operators_conj_transpose = np.transpose(np.conjugate(kraus_operators))
    proj = c - kraus_operators @ (kraus_operators @ np.transpose(np.conjugate(c)) + kraus_operators_conj_transpose @ c) / 2
    
    # Print debugging information
    print('c=', c)
    print('proj=', proj)
    
    # Return the updated Kraus operators
    return kraus_operators - proj"""

"def Derivative(rho, unitary_matrix, kraus_operators, epsilon=0.01, alpha=0.1):\n    # Deep copy of the Kraus operators to avoid unintended modifications\n    kraus_operators_plus = kraus_operators.copy()\n    kraus_operators_minus = kraus_operators.copy()\n    c = []\n    \n    # Loop through Kraus operators\n    for n, k in enumerate(kraus_operators):\n        k_plus = k + epsilon # Kj(+epsilon)\n        k_minus = k - epsilon # Kj(-epsilon)\n      \n        kraus_operators_plus[n] = k_plus # Replace Kj by new Kj_plus\n        kraus_operators_minus[n] = k_minus # Replace Kj by new Kj_minus\n\n        # Compute fidelity for K+ and K-\n        fil_plus = metric.compilation_trace_fidelity(rho, calRho3(rho=rho, unitary_matrix=unitary_matrix, kraus_operators=kraus_operators_plus))\n        fil_minus = metric.compilation_trace_fidelity(rho, calRho3(rho=rho, unitary_matrix=unitary_matrix, kraus_operators=kraus_operators_minus))\n        \n        # Compute the derivative\n        derivative 

In [960]:
#Auto Diff
def Derivative(rho, unitary_matrix, kraus_operators, n, alpha=0.1):
    tensorKraus = tf.Variable(kraus_operators) #4 Ks, each is a 4x4 matrix
    with tf.GradientTape() as tape:
        y = calRho3(rho, unitary_matrix, tensorKraus)
        f = 1 - compilation_trace_fidelity(rho, y) 
        #f = tf.convert_to_tensor(f)
    
    #print("RHO= ", rho)
    #print("F= ", f)
    # Get the gradient of y with respect to x
    c = tape.gradient(f, tensorKraus)

    #print("Function value:", f)
    

    # Ensure c is in the right shape and convert to numpy for further calculations
    c = tf.reshape(c, (2**n * 2**n,2**n))
    # Compute the conjugate transpose of the Kraus operators
    kraus_operators = tf.reshape(kraus_operators, (2**n * 2**n,2**n))
    #print('kraus_operators=\n', kraus_operators)
    
    # Compute the projection term
    proj = c - kraus_operators @ (np.transpose(np.conjugate(c)) @ kraus_operators  + np.transpose(np.conjugate(kraus_operators)) @ c) / 2

    # Update the Kraus operators
    updated_kraus_operators = kraus_operators - alpha * proj
    
    # Return the updated Kraus operators
    return updated_kraus_operators



In [969]:
KrausOperatorsTry = tf.identity(KrausOperators)

In [970]:
print(sum(K @ np.transpose(np.conjugate(K)) for K in KrausOperatorsTry))

tf.Tensor(
[[ 1.00000000e+00+0.j  1.11022302e-16+0.j  2.77555756e-17+0.j
   1.11022302e-16+0.j]
 [ 1.11022302e-16+0.j  1.00000000e+00+0.j  0.00000000e+00+0.j
   0.00000000e+00+0.j]
 [ 2.77555756e-17+0.j  0.00000000e+00+0.j  1.00000000e+00+0.j
  -2.22044605e-16+0.j]
 [ 1.11022302e-16+0.j  0.00000000e+00+0.j -2.22044605e-16+0.j
   1.00000000e+00+0.j]], shape=(4, 4), dtype=complex128)


In [971]:
# try looping manually
for i in range (0, 1000):
    a = cost(rho, calRho3(rho, unitary_matrix, KrausOperatorsTry))
    KrausOperatorsTry=Derivative(rho, unitary_matrix, KrausOperatorsTry, n)
    KrausOperatorsTry = tf.reshape(KrausOperatorsTry, (2**n,2**n,2**n))

    #print('K(t)K\n',sum(K @ np.transpose(np.conjugate(K)) for K in KrausOperatorsTry)) # I

    print (a)

    #print(KrausOperatorsTry) #4x4x4

tf.Tensor((0.45678794755065755+0j), shape=(), dtype=complex128)
tf.Tensor((0.4224170857395144+0j), shape=(), dtype=complex128)
tf.Tensor((0.390234579239513+0j), shape=(), dtype=complex128)
tf.Tensor((0.36018689620234423+0j), shape=(), dtype=complex128)
tf.Tensor((0.3322047556852381+0j), shape=(), dtype=complex128)
tf.Tensor((0.30620658333639583+0j), shape=(), dtype=complex128)
tf.Tensor((0.2821016693463392+0j), shape=(), dtype=complex128)
tf.Tensor((0.2597929957994777+0j), shape=(), dtype=complex128)
tf.Tensor((0.2391797157648473+0j), shape=(), dtype=complex128)
tf.Tensor((0.22015928064112822+0j), shape=(), dtype=complex128)
tf.Tensor((0.20262922443371445+0j), shape=(), dtype=complex128)
tf.Tensor((0.18648862328586158+0j), shape=(), dtype=complex128)
tf.Tensor((0.17163925558718107+0j), shape=(), dtype=complex128)
tf.Tensor((0.1579864924842247+0j), shape=(), dtype=complex128)
tf.Tensor((0.14543995092866457+0j), shape=(), dtype=complex128)
tf.Tensor((0.13391394191002048+0j), shape=(), dt

In [972]:
print(sum(K @ np.transpose(np.conjugate(K)) for K in KrausOperatorsTry))
print(KrausOperatorsTry)

tf.Tensor(
[[ 0.92321928+0.00000000e+00j  0.18385553+1.19944432e-18j
  -0.33749603+2.35803653e-20j  0.10649375+3.58094276e-19j]
 [ 0.18385553-1.19944432e-18j  1.10301302+0.00000000e+00j
  -0.32746435+6.76084691e-19j  0.1078828 -3.17984506e-19j]
 [-0.33749603-2.35803653e-20j -0.32746435-6.76084691e-19j
   1.01068411+0.00000000e+00j -0.18930421-2.08096569e-19j]
 [ 0.10649375-3.58094276e-19j  0.1078828 +3.17984506e-19j
  -0.18930421+2.08096569e-19j  0.96535657+0.00000000e+00j]], shape=(4, 4), dtype=complex128)
tf.Tensor(
[[[-0.25024378-4.99847339e-19j -0.39618767-4.19287528e-20j
    0.18977997+1.15045284e-19j -0.21487331-5.72782532e-20j]
  [-0.27219397+4.20038453e-19j  0.52868709-4.56065455e-20j
   -0.17956845+1.25136510e-19j  0.28146526-6.23024300e-20j]
  [ 0.13570247+2.81877720e-19j -0.26567752+2.27371700e-20j
    0.08891242-6.23868826e-20j -0.14137836+3.10609144e-20j]
  [-0.14760564+5.41579507e-21j  0.2868817 -2.47315655e-20j
   -0.09732287+6.78591606e-20j  0.15272548-3.37854287e-20j]]

In [976]:
rho_out_1 = sum(K @ rho @ np.transpose(np.conjugate(K)) for K in KrausOperatorsTry)
rho_out_2 = (unitary_matrix @ rho @ np.transpose(np.conjugate(unitary_matrix)))

In [980]:
print(compilation_trace_fidelity(rho_out_1, rho_out_2))

tf.Tensor((1+4.5236439652697204e-26j), shape=(), dtype=complex128)
